# 개요

- many-to-many
    - 대량의 말뭉치->소량의 말뭉치(요약)
- repo
    - gogamza/kobart-summarization
    - https://huggingface.co/gogamza/kobart-summarization

In [1]:
!pip install -q transformers

In [ ]:
!pip list

# 모델, 토크나이저 로드

In [ ]:
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [ ]:
repo  = 'gogamza/kobart-summarization'
tokenizer = PreTrainedTokenizerFast.from_pretrained( repo )
model = BartForConditionalGeneration.from_pretrained( repo )

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

# 문서 요약

In [ ]:
# 1. 입력 데이터
text = '''
(서울=뉴스1) 김정현 기자 = 현대차(005380)가 수소밸류 체인 관련 비전을 제시하면서 수소차 관련주가 장중 강세다.
한국거래소에 따르면 22일 낮 12시31분 기준 두산퓨얼셀(336260)은 전일 대비 3550원(17.36%) 오른 2만 4000원에 거래 중이다.
이외에도 △두산퓨얼셀우 27.88% △한화솔루션(009830) 4.99% △범한퓨얼셀(382900) 7.51% △에스퓨얼셀(288620) 7.27% 등 수소차·수소전지 관련주들도 강세다.
이같은 수소차 관련주의 상승세는 현대차가 수소 상용 밸류체인 확장 전략을 발표한 영향으로 풀이된다.
현대차는 미국 라스베이거스 컨벤션센터(LVCC)에서 열리는 청정 운송수단 박람회 'ACT 엑스포 2024'에 참가해 '엑시언트 수소전기트럭' 상품성 개선 콘셉트 모델을 공개했다.
또 △수소 공급 및 충전소 구축 △리스 및 파이낸싱 △유지보수 서비스 등 '수소 상용 모빌리티 밸류체인'을 본격적으로 구축한다는 게획도 밝혔다.
'''
text = text.strip()

In [ ]:
# 2. 인코딩
text_vec = tokenizer.encode( text )
print( text_vec )
# 토큰을 단순하게 수치로 변환한 데이터, 문장 표시, 중요도에 관한 데이터도 추가로 존재

[21224, 22156, 17532, 13672, 14680, 15146, 17358, 239, 14079, 252, 250, 17918, 15092, 14032, 11319, 10802, 10428, 14344, 12037, 14342, 27731, 15299, 14457, 14032, 11319, 12612, 14342, 16387, 14137, 12273, 14119, 20446, 15274, 15471, 16755, 14655, 16670, 14793, 26359, 25756, 10952, 14832, 17712, 13536, 11771, 11291, 15377, 24418, 249, 17500, 15137, 27408, 15171, 17430, 15894, 11936, 27148, 15625, 253, 16705, 15830, 18078, 26538, 15288, 15337, 16324, 12034, 11890, 14279, 15451, 9925, 11211, 13536, 11771, 11291, 11911, 15433, 16030, 255, 236, 15451, 13590, 13714, 11326, 20600, 239, 14079, 25292, 15708, 240, 14136, 16157, 256, 236, 15451, 10839, 13590, 13536, 11771, 11291, 15377, 255, 21065, 14079, 240, 14235, 15318, 29867, 15451, 24709, 13536, 11771, 11291, 15018, 23534, 253, 14745, 240, 14235, 15478, 254, 236, 14048, 14032, 11319, 12612, 27258, 11319, 26272, 14342, 12258, 15038, 14119, 20446, 12034, 16044, 14032, 11319, 12612, 14342, 14453, 14919, 18445, 14689, 17010, 14032, 11319, 25053

In [ ]:
# 3. 요약 -> bart 형식에 맞게 스페셜 토큰 부여

# 3-1. 해당 토크나이저는 문장의 시작과 끝에 각각 스페셜 토큰을 부여하여 진행
tokenizer.bos_token, tokenizer.eos_token

('<s>', '</s>')

In [ ]:
# 3-2. 문장 백터의 앞뒤에 스페셜 토큰 추가
input_vec = [tokenizer.bos_token_id] + text_vec + [tokenizer.eos_token_id]
print( input_vec )

[0, 21224, 22156, 17532, 13672, 14680, 15146, 17358, 239, 14079, 252, 250, 17918, 15092, 14032, 11319, 10802, 10428, 14344, 12037, 14342, 27731, 15299, 14457, 14032, 11319, 12612, 14342, 16387, 14137, 12273, 14119, 20446, 15274, 15471, 16755, 14655, 16670, 14793, 26359, 25756, 10952, 14832, 17712, 13536, 11771, 11291, 15377, 24418, 249, 17500, 15137, 27408, 15171, 17430, 15894, 11936, 27148, 15625, 253, 16705, 15830, 18078, 26538, 15288, 15337, 16324, 12034, 11890, 14279, 15451, 9925, 11211, 13536, 11771, 11291, 11911, 15433, 16030, 255, 236, 15451, 13590, 13714, 11326, 20600, 239, 14079, 25292, 15708, 240, 14136, 16157, 256, 236, 15451, 10839, 13590, 13536, 11771, 11291, 15377, 255, 21065, 14079, 240, 14235, 15318, 29867, 15451, 24709, 13536, 11771, 11291, 15018, 23534, 253, 14745, 240, 14235, 15478, 254, 236, 14048, 14032, 11319, 12612, 27258, 11319, 26272, 14342, 12258, 15038, 14119, 20446, 12034, 16044, 14032, 11319, 12612, 14342, 14453, 14919, 18445, 14689, 17010, 14032, 11319, 25

In [ ]:
# 3-3 요약 생성
import torch

summary_ids = model.generate( torch.tensor( [input_vec] ),
                max_new_tokens = 40 # 요약하는 최대 토큰수
              )

print( summary_ids )

tensor([[    2, 17358,   239, 14032, 11319, 12612,   240, 14032, 11319, 12612,
         14342, 16387, 14032, 11319, 25053, 23971, 10428, 27245, 18391, 20042,
         17411, 21808, 14137, 12273, 14119, 27543, 14557, 15615,     1]])


In [ ]:
# 최대 40이므로, 40이하로 요약 진행
summary_ids.size()

torch.Size([1, 29])

In [ ]:
# 3-4 디코딩 확인
# [2, ..... , 1 ] => 2, 1은 스페셜 토큰
print( tokenizer.decode( summary_ids.squeeze().tolist(), skip_special_tokens=True ) )

현대차( 수소차) 수소차 관련주가 수소 상용 밸류체인 확장 전략을 발표한 영향으로 장중 강폭을 기록했다.
